# This notebook prepares the primary data sets and visits some tips and techniques along the way

* Full data pulled from retrosheet.org
* Chadwick tools used for converting retrosheet data http://chadwick.sourceforge.net/doc/cwtools.html

We'll be using Retrosheet baseball data for our examples in this discussion.  At the center of the work will be the Retrosheet 'Event Files' from 2017.  Each row in the event file describes some kind of action on the field. Any time the game situation is different from the previous pitch, one or more events is created.  The events are rich in data.  They indicate the game, the inning, the batter, the type of hit, the direction of the hit, the number of runners on base etc. We'll take a look at the event codes a bit later to get some impression of the depth of information.  There are 191,996 events recorded for 2017.  There were 81\*30 or 2430 scheduled games in 2017. This yields, on average, about 79 events per game.

In [120]:
import pandas as pd
import os
import matplotlib

# % and %% are ipthyon 'magics'
# ! is ipython's shell execution shortcut

In [121]:
%history

import pandas as pd
import os
import matplotlib
%history
!ls data_public/*.EV*
%%sh
head -3 data_public/2017CHA.EVA
echo ""
echo 'data_public/2017CHA.EVA'
echo ""
sort -k2 -t, data_public/2017CHA.EVA |head -5
files = !ls
type(files)
files.grep('\.i.*')
files.p
extension = 'ipynbb'
!ls *.{extension}
!ls *.{extension[0:-1]}
#Chadwick expects a 'team' file in the cwd
!ln -s ./data_public/TEAM2017 team
! cwevent -n data_public/2017SEA.EVA |head -1 >data_public/atbats.txt
%%sh
for x in $(ls -1 data_public/*.EV*); do cwevent $x >>data_public/atbats.txt; done
cat data_public/*.ROS >data_public/rosters.txt
event_code = {}
with open("data_public/event_codes.txt") as f:
    for line in f:
       (val, key) = line.split()
       event_code[key] = int(val)
event_code
df_events=pd.read_csv('./data_public/atbats.txt')
df_players=pd.read_csv('./data_public/players.txt')
df_events.shape
df_events.columns
df_events.head()
df_events.info()
df_events.describe()
df_events['GAME_ID'].head()
df_events[['GAM

In [122]:
!ls data_public//*.EV*

data_public//2017ANA.EVA data_public//2017DET.EVA data_public//2017PHI.EVN
data_public//2017ARI.EVN data_public//2017HOU.EVA data_public//2017PIT.EVN
data_public//2017ATL.EVN data_public//2017KCA.EVA data_public//2017SDN.EVN
data_public//2017BAL.EVA data_public//2017LAN.EVN data_public//2017SEA.EVA
data_public//2017BOS.EVA data_public//2017MIA.EVN data_public//2017SFN.EVN
data_public//2017CHA.EVA data_public//2017MIL.EVN data_public//2017SLN.EVN
data_public//2017CHN.EVN data_public//2017MIN.EVA data_public//2017TBA.EVA
data_public//2017CIN.EVN data_public//2017NYA.EVA data_public//2017TEX.EVA
data_public//2017CLE.EVA data_public//2017NYN.EVN data_public//2017TOR.EVA
data_public//2017COL.EVN data_public//2017OAK.EVA data_public//2017WAS.EVN


In [123]:
%%sh
head -3 data_public/2017CHA.EVA
echo ""
echo 'data_public/2017CHA.EVA'
echo ""
sort -k2 -t, data_public/2017CHA.EVA |head -5

id,CHA201704040
version,2
info,visteam,DET

data_public/2017CHA.EVA

com," - batter out at 2nd"
com,"$Anderson beat throw to first on close play; White Sox"
com,"$Angels challenged call that Phillips was out on fly ball;"
com,"$Astros challenged call that Garcia was hit by pitch;"
com,"$Athletics challenged call of out at first; call"


#### Useful techniques for interacting with the shell

* <b>Use assignment to capture the ouput of your ! command</b>

In [124]:
files = !ls

In [125]:
type(files)

IPython.utils.text.SList

In [126]:
files.grep('\.i.*')

['Baseball.ipynb',
 'Examples.ipynb',
 'Introl to Jupyter.ipynb',
 'Jupyter-intro-data.ipynb',
 'Jupyter-intro.ipynb',
 'Mxw_cmp_Active_not_Sync.ipynb',
 'Salesforce_Demo.ipynb',
 'Untitled.ipynb',
 'Untitled1.ipynb',
 'crw_stage_for_deletion.ipynb',
 'example_json.ipynb',
 'stripped.ipynb']

In [127]:
files.p

[PosixPath('Baseball.ipynb'),
 PosixPath('Contacts_EventDetails.py'),
 PosixPath('Examples.ipynb'),
 PosixPath('Introl to Jupyter.ipynb'),
 PosixPath('Jupyter-intro-data.ipynb'),
 PosixPath('Jupyter-intro.ipynb'),
 PosixPath('Mxw_cmp_Active_not_Sync.ipynb'),
 PosixPath('Report_test.py'),
 PosixPath('Salesforce_Demo.ipynb'),
 PosixPath('Untitled.ipynb'),
 PosixPath('Untitled1.ipynb'),
 PosixPath('__pycache__'),
 PosixPath('boimetrics_core_taskflow.txt'),
 PosixPath('crw_stage_for_deletion.ipynb'),
 PosixPath('data'),
 PosixPath('data_public'),
 PosixPath('example_json.ipynb'),
 PosixPath('notebook'),
 PosixPath('requirements.txt'),
 PosixPath('requirements.txt.20181010'),
 PosixPath('requirements_pre-jupyter.txt'),
 PosixPath('salesforce'),
 PosixPath('settings.py'),
 PosixPath('sfdc'),
 PosixPath('stripped.ipynb'),
 PosixPath('team'),
 PosixPath('vp_utils.py'),
 PosixPath('x.x')]

## Passing Python output to the shell
#### Here we'll do the opposite and more powerful technique
* <b>We'll use Use {expression} to pass from ipython to the shell</b>

In [128]:
extension = 'ipynbb'

In [129]:
!ls *.{extension}

ls: *.ipynbb: No such file or directory


* Hmm... Extra trailing 'b'. Lets take a slice of the extension string

In [130]:
!ls *.{extension[0:-1]}

Baseball.ipynb                Salesforce_Demo.ipynb
Examples.ipynb                Untitled.ipynb
Introl to Jupyter.ipynb       Untitled1.ipynb
Jupyter-intro-data.ipynb      crw_stage_for_deletion.ipynb
Jupyter-intro.ipynb           example_json.ipynb
Mxw_cmp_Active_not_Sync.ipynb stripped.ipynb


### This shows us a few techniques and examples. <br> In the following section, we'll use these techniques to bring in the data set that we'll use for our analysis.

### Here we're preparing the file.  Run the cwevent executable with -n and capture the header

In [131]:
#Chadwick expects a 'team' file in the cwd
!ln -s ./data_public/TEAM2017 team

ln: team: File exists


In [132]:
! cwevent -n data_public/2017SEA.EVA |head -1 >data_public/atbats.txt


Chadwick expanded event descriptor, version 0.7.1
  Type 'cwevent -h' for help.
Copyright (c) 2002-2018
Dr T L Turocy, Chadwick Baseball Bureau (ted.turocy@gmail.com)
This is free software, subject to the terms of the GNU GPL license.

[Processing file data_public/2017SEA.EVA.]


### Now we'll shell out and run a loop to invoke the converter on each event file.  We'll also concatentate the roster files in a separate command

In [133]:
%%sh
for x in $(ls -1 data_public/*.EV*); do cwevent $x >>data_public/atbats.txt; done
cat data_public/*.ROS >data_public/rosters.txt


Chadwick expanded event descriptor, version 0.7.1
  Type 'cwevent -h' for help.
Copyright (c) 2002-2018
Dr T L Turocy, Chadwick Baseball Bureau (ted.turocy@gmail.com)
This is free software, subject to the terms of the GNU GPL license.

[Processing file data_public/2017ANA.EVA.]

Chadwick expanded event descriptor, version 0.7.1
  Type 'cwevent -h' for help.
Copyright (c) 2002-2018
Dr T L Turocy, Chadwick Baseball Bureau (ted.turocy@gmail.com)
This is free software, subject to the terms of the GNU GPL license.

[Processing file data_public/2017ARI.EVN.]

Chadwick expanded event descriptor, version 0.7.1
  Type 'cwevent -h' for help.
Copyright (c) 2002-2018
Dr T L Turocy, Chadwick Baseball Bureau (ted.turocy@gmail.com)
This is free software, subject to the terms of the GNU GPL license.

[Processing file data_public/2017ATL.EVN.]

Chadwick expanded event descriptor, version 0.7.1
  Type 'cwevent -h' for help.
Copyright (c) 2002-2018
Dr T L Turocy, Chadwick Baseball Bureau (ted.turocy@gma

### Standard Python file to dictionary
* Constants for event codes
* Here we're reading a two column file into a Python key:val (dictionary) data structure
* This allows us to use '2B' and 'HR' vs 21 and 23 to identify event rows of interest

In [134]:
event_code = {}
with open("data_public/event_codes.txt") as f:
    for line in f:
       (val, key) = line.split()
       event_code[key] = int(val)

In [135]:
event_code

{'UNK': 0,
 'NONE': 1,
 'GENERIC_OUT': 2,
 'K': 3,
 'SB': 4,
 'DEF_INDIFFERENCE': 5,
 'SB_CAUGHT': 6,
 'ERROR_PICKOFF': 7,
 'PICKOFF': 8,
 'WP': 9,
 'PB': 10,
 'BK': 11,
 'OTHER_ADVANCE': 12,
 'ERROR_FOUL': 13,
 'BB': 14,
 'IBB': 15,
 'HBP': 16,
 'INTERFERENCE': 17,
 'ERROR': 18,
 'FC': 19,
 'SINGLE': 20,
 'DOUBLE': 21,
 'TRIPLE': 22,
 'HR': 23,
 'MISSING': 24}

In [136]:
df_events=pd.read_csv('./data_public/atbats.txt')
df_players=pd.read_csv('./data_public/players.txt')

### Here are a few Pandas tools for getting an overview of a dataframe

In [137]:
df_events.shape

(191196, 36)

In [138]:
df_events.columns

Index(['GAME_ID', 'AWAY_TEAM_ID', 'INN_CT', 'BAT_HOME_ID', 'OUTS_CT',
       'BALLS_CT', 'STRIKES_CT', 'AWAY_SCORE_CT', 'HOME_SCORE_CT',
       'RESP_BAT_ID', 'RESP_BAT_HAND_CD', 'RESP_PIT_ID', 'RESP_PIT_HAND_CD',
       'BASE1_RUN_ID', 'BASE2_RUN_ID', 'BASE3_RUN_ID', 'EVENT_TX',
       'LEADOFF_FL', 'PH_FL', 'BAT_FLD_CD', 'BAT_LINEUP_ID', 'EVENT_CD',
       'BAT_EVENT_FL', 'AB_FL', 'H_CD', 'SH_FL', 'SF_FL', 'EVENT_OUTS_CT',
       'RBI_CT', 'WP_FL', 'PB_FL', 'ERR_CT', 'BAT_DEST_ID', 'RUN1_DEST_ID',
       'RUN2_DEST_ID', 'RUN3_DEST_ID'],
      dtype='object')

In [139]:
df_events.head()

,GAME_ID,AWAY_TEAM_ID,INN_CT,BAT_HOME_ID,OUTS_CT,BALLS_CT,STRIKES_CT,AWAY_SCORE_CT,HOME_SCORE_CT,RESP_BAT_ID,...,SF_FL,EVENT_OUTS_CT,RBI_CT,WP_FL,PB_FL,ERR_CT,BAT_DEST_ID,RUN1_DEST_ID,RUN2_DEST_ID,RUN3_DEST_ID
0,ANA201704070,SEA,1,0,0,3,2,0,0,seguj002,...,F,0,0,F,F,0,1,0,0,0
1,ANA201704070,SEA,1,0,0,1,2,0,0,hanim001,...,F,1,0,F,F,0,0,1,0,0
2,ANA201704070,SEA,1,0,1,1,1,0,0,canor001,...,F,1,0,F,F,0,0,1,0,0
3,ANA201704070,SEA,1,0,2,0,1,0,0,cruzn002,...,F,0,0,F,F,0,0,2,0,0
4,ANA201704070,SEA,1,0,2,2,2,0,0,cruzn002,...,F,1,0,F,F,0,0,0,2,0


In [140]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191196 entries, 0 to 191195
Data columns (total 36 columns):
GAME_ID             191196 non-null object
AWAY_TEAM_ID        191196 non-null object
INN_CT              191196 non-null int64
BAT_HOME_ID         191196 non-null int64
OUTS_CT             191196 non-null int64
BALLS_CT            191196 non-null int64
STRIKES_CT          191196 non-null int64
AWAY_SCORE_CT       191196 non-null int64
HOME_SCORE_CT       191196 non-null int64
RESP_BAT_ID         191196 non-null object
RESP_BAT_HAND_CD    191196 non-null object
RESP_PIT_ID         191196 non-null object
RESP_PIT_HAND_CD    191196 non-null object
BASE1_RUN_ID        61225 non-null object
BASE2_RUN_ID        37543 non-null object
BASE3_RUN_ID        19364 non-null object
EVENT_TX            191196 non-null object
LEADOFF_FL          191196 non-null object
PH_FL               191196 non-null object
BAT_FLD_CD          191196 non-null int64
BAT_LINEUP_ID       191196 non-null int6

In [141]:
df_events.describe()

,INN_CT,BAT_HOME_ID,OUTS_CT,BALLS_CT,STRIKES_CT,AWAY_SCORE_CT,HOME_SCORE_CT,BAT_FLD_CD,BAT_LINEUP_ID,EVENT_CD,H_CD,EVENT_OUTS_CT,RBI_CT,ERR_CT,BAT_DEST_ID,RUN1_DEST_ID,RUN2_DEST_ID,RUN3_DEST_ID
count,191196.000000,191196.000000,191196.000000,191196.000000,191196.000000,191196.000000,191196.000000,191196.000000,191196.000000,191196.000000,191196.000000,191196.000000,191196.000000,191196.000000,191196.000000,191196.000000,191196.000000,191196.000000
mean,4.993143,0.491255,0.980821,1.319803,1.290179,2.437467,2.321879,5.812663,4.825823,7.779472,0.368820,0.678733,0.112753,0.014734,0.496433,0.465596,0.494315,0.339662
std,2.640043,0.499925,0.816872,1.094033,0.807875,2.664073,2.671078,2.666287,2.573597,7.847862,0.853667,0.514235,0.407539,0.121909,0.873902,0.883687,1.073469,1.040767
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,4.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.000000,0.000000,1.000000,1.000000,2.000000,2.000000,1.000000,6.000000,5.000000,3.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.000000,1.000000,2.000000,2.000000,2.000000,4.000000,4.000000,8.000000,7.000000,14.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
max,19.000000,1.000000,2.000000,3.000000,2.000000,22.000000,23.000000,11.000000,9.000000,23.000000,4.000000,3.000000,4.000000,2.000000,6.000000,6.000000,6.000000,6.000000


### Now we'll look at a few pandas techniques
* First we'll restrict the dataframe to a single column
* Next we'll restrict the dataframe to a set of columns
* Third we'll breakdown the contents of a column
* Fourth we'll use value_counts() to get a summary

In [142]:
df_events['GAME_ID'].head()

0    ANA201704070
1    ANA201704070
2    ANA201704070
3    ANA201704070
4    ANA201704070
Name: GAME_ID, dtype: object

* Notice that we restrict with dataframe[] and provide a python list of the colums with ['item1','item2'....] resulting in doubling of the brackets

In [143]:
df_events[['GAME_ID','AWAY_TEAM_ID','BALLS_CT','RESP_BAT_ID','OUTS_CT','EVENT_CD']].head()

,GAME_ID,AWAY_TEAM_ID,BALLS_CT,RESP_BAT_ID,OUTS_CT,EVENT_CD
0,ANA201704070,SEA,3,seguj002,0,14
1,ANA201704070,SEA,1,hanim001,0,3
2,ANA201704070,SEA,1,canor001,1,2
3,ANA201704070,SEA,0,cruzn002,2,4
4,ANA201704070,SEA,2,cruzn002,2,3


In [144]:
df_events['GAME_ID'].str[0:3].head()

0    ANA
1    ANA
2    ANA
3    ANA
4    ANA
Name: GAME_ID, dtype: object

Below we apply the series.value_counts( ) method to return a series whose index is each distinct value 

In [145]:
df_events['GAME_ID'].str[0:3].value_counts()

DET    6547
BOS    6542
TEX    6530
MIN    6525
CHN    6483
BAL    6471
ATL    6440
ARI    6439
OAK    6435
SFN    6433
CIN    6433
MIL    6426
COL    6420
PIT    6411
PHI    6407
MIA    6403
WAS    6370
NYN    6369
CHA    6336
SLN    6327
SEA    6325
NYA    6320
HOU    6310
TBA    6290
TOR    6248
KCA    6244
ANA    6230
CLE    6185
LAN    6169
SDN    6128
Name: GAME_ID, dtype: int64

### Let's look at the data quickly to get a bit of a better idea of what's in the files

In [146]:
df_events[['GAME_ID','AWAY_TEAM_ID','BALLS_CT','RESP_BAT_ID','OUTS_CT','EVENT_CD']].head()

,GAME_ID,AWAY_TEAM_ID,BALLS_CT,RESP_BAT_ID,OUTS_CT,EVENT_CD
0,ANA201704070,SEA,3,seguj002,0,14
1,ANA201704070,SEA,1,hanim001,0,3
2,ANA201704070,SEA,1,canor001,1,2
3,ANA201704070,SEA,0,cruzn002,2,4
4,ANA201704070,SEA,2,cruzn002,2,3


* Note above that there are two consecutive events for the same batter (cruzn002).  We'll use the pandas indexed lookup method (.iloc) to convert a row of the dataframe into a series.
* The series has index that is the column name values that are the data from that row.
* Event codes here are 14=walk, 3=K, 2=generic out and 4=stolen base

In [147]:
df_events.iloc[3]

GAME_ID             ANA201704070
AWAY_TEAM_ID                 SEA
INN_CT                         1
BAT_HOME_ID                    0
OUTS_CT                        2
BALLS_CT                       0
STRIKES_CT                     1
AWAY_SCORE_CT                  0
HOME_SCORE_CT                  0
RESP_BAT_ID             cruzn002
RESP_BAT_HAND_CD               ?
RESP_PIT_ID             chavj001
RESP_PIT_HAND_CD               ?
BASE1_RUN_ID            seguj002
BASE2_RUN_ID                 NaN
BASE3_RUN_ID                 NaN
EVENT_TX                     SB2
LEADOFF_FL                     F
PH_FL                          F
BAT_FLD_CD                    10
BAT_LINEUP_ID                  4
EVENT_CD                       4
BAT_EVENT_FL                   F
AB_FL                          F
H_CD                           0
SH_FL                          F
SF_FL                          F
EVENT_OUTS_CT                  0
RBI_CT                         0
WP_FL                          F
PB_FL     

In [148]:
df_events.iloc[3].loc['EVENT_CD']

4

In [149]:
df_events['AWAY_TEAM_ID'].value_counts()

TOR    6579
NYN    6526
HOU    6521
MIA    6520
NYA    6512
CIN    6499
ATL    6473
BOS    6420
KCA    6394
SEA    6392
SDN    6385
SLN    6377
SFN    6361
TBA    6359
CHA    6355
COL    6353
CHN    6351
MIL    6351
LAN    6326
MIN    6326
PHI    6323
ANA    6312
PIT    6299
DET    6278
OAK    6277
WAS    6276
ARI    6275
CLE    6266
BAL    6265
TEX    6245
Name: AWAY_TEAM_ID, dtype: int64

In [150]:
df_events['AWAY_TEAM_ID'].value_counts().index

Index(['TOR', 'NYN', 'HOU', 'MIA', 'NYA', 'CIN', 'ATL', 'BOS', 'KCA', 'SEA',
       'SDN', 'SLN', 'SFN', 'TBA', 'CHA', 'COL', 'CHN', 'MIL', 'LAN', 'MIN',
       'PHI', 'ANA', 'PIT', 'DET', 'OAK', 'WAS', 'ARI', 'CLE', 'BAL', 'TEX'],
      dtype='object')

#### This shows us a few techniques and examples. <br> In the following section, we'll use these techniques to bring in the data set that we'll use for our analysis.

#### Using value_counts to inspect a column
* Here we use value_counts to generate list of entities from a series 
* Functionally identical to "<b> cat file |cut -d, -f4|sort|uniq -c |sort -rn</b>"
* Functionally identical to "<b>select column, count(column) group by column order by count(column)</b>")
* value_counts() is  a series or index method. It returns a series 
* Note again that the index of the series is the value in the column

In [151]:
df_events['AWAY_TEAM_ID'].value_counts()

TOR    6579
NYN    6526
HOU    6521
MIA    6520
NYA    6512
CIN    6499
ATL    6473
BOS    6420
KCA    6394
SEA    6392
SDN    6385
SLN    6377
SFN    6361
TBA    6359
CHA    6355
COL    6353
CHN    6351
MIL    6351
LAN    6326
MIN    6326
PHI    6323
ANA    6312
PIT    6299
DET    6278
OAK    6277
WAS    6276
ARI    6275
CLE    6266
BAL    6265
TEX    6245
Name: AWAY_TEAM_ID, dtype: int64

#### Create a few filters to restrict rows (filter is not a defined Pandas term)
* These types of restrictions return a pandas series
* We've named the series reflecting the filter(s) applied 
* Each series is a set of booleans indexed identically to the source

In [152]:
flt_homers = df_events['EVENT_CD'] == event_code['HR']
flt_redsox = (df_events['GAME_ID'].str.startswith('BOS')) | (df_events['AWAY_TEAM_ID'] == 'BOS')
flt_yankees = (df_events['GAME_ID'].str.startswith('NYA')) | (df_events['AWAY_TEAM_ID'] == 'NYA')

In [153]:
flt_homers.value_counts()

False    185091
True       6105
Name: EVENT_CD, dtype: int64

In [154]:
flt_homers.head()

0    False
1    False
2    False
3    False
4    False
Name: EVENT_CD, dtype: bool

In [155]:
flt_redsox.value_counts()

False    178234
True      12962
dtype: int64

### Using an index along with python looping
* An index is an iterable object
* We traverse it here to make dictionaries of filters
* First we'll show the index, then we'll iterate to create more filters

In [156]:
df_events['AWAY_TEAM_ID'].value_counts().index

Index(['TOR', 'NYN', 'HOU', 'MIA', 'NYA', 'CIN', 'ATL', 'BOS', 'KCA', 'SEA',
       'SDN', 'SLN', 'SFN', 'TBA', 'CHA', 'COL', 'CHN', 'MIL', 'LAN', 'MIN',
       'PHI', 'ANA', 'PIT', 'DET', 'OAK', 'WAS', 'ARI', 'CLE', 'BAL', 'TEX'],
      dtype='object')

In [157]:
flt_home_team = {}
flt_away_team = {}
for team in df_events['AWAY_TEAM_ID'].value_counts().index:
#for team in ['BOS']:
    #print(type(team))
    flt_home_team[team] = (df_events['GAME_ID'].str.startswith(team)) & (df_events['BAT_HOME_ID'] == 1)
    flt_away_team[team] = (df_events['AWAY_TEAM_ID'] == team) & (df_events['BAT_HOME_ID'] == 0)

In [158]:
df_events[flt_away_team['DET']].shape

(3119, 36)

* Our first practical question... How many doubles did the Red Sox have in 2017? 

In [159]:
df_events[((flt_home_team['BOS']) | (flt_away_team['BOS'])) \
          & (df_events['EVENT_CD'] == event_code['DOUBLE'])].shape

(302, 36)

* Who had them??

In [160]:
df_events[((flt_home_team['BOS']) | (flt_away_team['BOS'])) \
          & (df_events['EVENT_CD'] == event_code['DOUBLE'])].head()

#[['GAME_ID','OUTS_CT','BALLS_CT', 'STRIKES_CT', 'AWAY_SCORE_CT', 'HOME_SCORE_CT','RESP_BAT_ID']]


,GAME_ID,AWAY_TEAM_ID,INN_CT,BAT_HOME_ID,OUTS_CT,BALLS_CT,STRIKES_CT,AWAY_SCORE_CT,HOME_SCORE_CT,RESP_BAT_ID,...,SF_FL,EVENT_OUTS_CT,RBI_CT,WP_FL,PB_FL,ERR_CT,BAT_DEST_ID,RUN1_DEST_ID,RUN2_DEST_ID,RUN3_DEST_ID
3655,ANA201707210,BOS,1,0,0,3,1,0,0,bettm001,...,F,0,0,F,F,0,2,0,0,0
3662,ANA201707210,BOS,1,0,1,0,1,4,0,bradj001,...,F,1,1,F,F,0,0,4,0,0
3745,ANA201707220,BOS,2,0,2,2,2,1,0,bettm001,...,F,0,1,F,F,0,2,0,0,5
3782,ANA201707220,BOS,6,0,2,2,2,3,6,bogax001,...,F,0,0,F,F,0,2,0,0,0
19712,BAL201704230,BOS,8,0,0,1,1,6,0,vazqc001,...,F,0,0,F,F,0,2,0,0,0


In [161]:
df_events[((flt_home_team['BOS']) | (flt_away_team['BOS'])) \
          & (df_events['EVENT_CD'] == event_code['DOUBLE'])]['RESP_BAT_ID'].value_counts()

bettm001    46
morem001    34
bogax001    32
benia002    26
ramih003    24
pedrd001    19
bradj001    19
vazqc001    18
dever001    14
leons001    14
younc004    12
nunee002    12
marrd001     9
travs001     6
holtb002     6
hernm003     3
sandp001     2
davir003     2
rutlj001     2
selss001     1
salec001     1
Name: RESP_BAT_ID, dtype: int64

## Pandas Merging Data Sets
* Let's revisit the Red Sox Doubles

In [162]:
type(df_events[((flt_home_team['BOS']) | (flt_away_team['BOS'])) \
          & (df_events['EVENT_CD'] == event_code['DOUBLE'])]['RESP_BAT_ID'])

pandas.core.series.Series

In [163]:
type(df_events[((flt_home_team['BOS']) | (flt_away_team['BOS'])) \
          & (df_events['EVENT_CD'] == event_code['DOUBLE'])])

pandas.core.frame.DataFrame

* Pandas merge( ) is a dataframe method so we'll have to use our dataframe version of the result set

In [164]:
df_players.head(3)

,ID,Last,First,Player debut,Mgr debut,Coach debut,Ump debut
0,aardd001,Aardsma,David,04/06/2004,NaN,NaN,NaN
1,aaroh101,Aaron,Hank,04/13/1954,NaN,NaN,NaN
2,aarot101,Aaron,Tommie,04/10/1962,NaN,04/06/1979,NaN


In [165]:
df_events[((flt_home_team['BOS']) | (flt_away_team['BOS'])) \
          & (df_events['EVENT_CD'] == event_code['DOUBLE'])]['RESP_BAT_ID'].value_counts()

bettm001    46
morem001    34
bogax001    32
benia002    26
ramih003    24
pedrd001    19
bradj001    19
vazqc001    18
dever001    14
leons001    14
younc004    12
nunee002    12
marrd001     9
travs001     6
holtb002     6
hernm003     3
sandp001     2
davir003     2
rutlj001     2
selss001     1
salec001     1
Name: RESP_BAT_ID, dtype: int64

In [166]:
df_events[((flt_home_team['BOS']) | (flt_away_team['BOS'])) \
          & (df_events['EVENT_CD'] == event_code['DOUBLE'])]\
.merge(df_players,left_on = 'RESP_BAT_ID', right_on = 'ID')['Last'].value_counts()

Betts         46
Moreland      34
Bogaerts      32
Benintendi    26
Ramirez       24
Bradley       19
Pedroia       19
Vazquez       18
Devers        14
Leon          14
Nunez         12
Young         12
Marrero        9
Travis         6
Holt           6
Hernandez      3
Davis          2
Sandoval       2
Rutledge       2
Selsky         1
Sale           1
Name: Last, dtype: int64

In [167]:
old_cols=set(df_events.columns)
df_events_players = pd.DataFrame(df_events.merge(df_players[['ID','Last','First','Player debut']]
                            , left_on='RESP_BAT_ID',right_on='ID', validate = 'm:1'))

In [168]:
new_cols=set(df_events_players.columns)

In [169]:
new_cols-old_cols

{'First', 'ID', 'Last', 'Player debut'}

## Pandas Grouping and Aggregation

In [170]:
grp_batter = df_events.groupby(['RESP_BAT_ID'])

In [173]:
grp_hits=df_events.groupby([df_events['RESP_BAT_ID'],df_events[df_events['EVENT_CD'].isin(
    [event_code['SINGLE'],event_code['DOUBLE'],event_code['TRIPLE'],event_code['HR']])]['EVENT_CD']])

In [174]:
grp_hits['RESP_BAT_ID'].count()

RESP_BAT_ID  EVENT_CD
abrej003     20.0        107
             21.0         43
             22.0          6
             23.0         33
adaml001     20.0         20
             21.0          4
             22.0          1
             23.0          5
adamm002     20.0         50
             21.0         22
             22.0          1
             23.0         20
adduj002     20.0         11
             21.0          6
             22.0          2
             23.0          1
adlet001     20.0          2
             21.0          1
adrie001     20.0         30
             21.0          9
             22.0          2
             23.0          2
aguij001     20.0         41
             21.0         15
             22.0          2
             23.0         16
ahmen001     20.0         27
             21.0          8
             22.0          1
             23.0          6
                        ... 
woodt004     20.0          2
             23.0          2
worlv001     20.0    

In [175]:
df_events[df_events['EVENT_CD'].isin(
    [event_code['SINGLE'],event_code['DOUBLE'],event_code['TRIPLE'],event_code['HR']])]['EVENT_CD']

5         20
6         21
16        20
20        20
21        20
32        20
38        20
43        20
44        20
45        20
47        23
48        20
51        20
56        20
57        23
65        20
73        20
74        23
84        21
86        20
88        20
91        20
93        21
95        20
98        20
105       20
107       21
112       23
115       20
121       20
          ..
191095    20
191101    21
191102    20
191104    21
191107    23
191113    20
191116    20
191120    20
191125    20
191127    21
191137    20
191140    21
191141    20
191142    20
191147    20
191151    21
191152    20
191155    20
191157    20
191167    23
191170    20
191171    21
191173    20
191174    20
191176    20
191180    20
191182    20
191184    21
191190    20
191194    20
Name: EVENT_CD, Length: 42215, dtype: int64

In [176]:
df_events[df_events['EVENT_CD'].isin(
    [event_code['SINGLE'],event_code['DOUBLE'],event_code['TRIPLE'],event_code['HR']])]['RESP_BAT_ID']

5         escoy001
6         calhk001
16        zunim001
20        maldm001
21        escoy001
32        maldm001
38        troum001
43        hanim001
44        canor001
45        cruzn002
47        maybc001
48        simma001
51        valed001
56        escoy001
57        calhk001
65        simma001
73        seguj002
74        hanim001
84        pujoa001
86        maybc001
88        espid001
91        seguj002
93        escoy001
95        calhk001
98        simma001
105       maldm001
107       valed001
112       escoy001
115       pujoa001
121       martj007
            ...   
191095    bostc001
191101    morom001
191102    stalj001
191104    turnt001
191107    renda001
191113    marts002
191116    wietm001
191120    harpb003
191125    murpd006
191127    wertj001
191137    murpd006
191140    mccua001
191141    bellj005
191142    freed001
191147    lobaj001
191151    stalj001
191152    polag001
191155    rodrs002
191157    sanca007
191167    taylm002
191170    harpb003
191171    sa

### Crosstabs
* First we'll look at players by EventCode
* Next we'll only pass the EventCodes of interest into the crosstab

In [177]:
pd.crosstab(df_events['RESP_BAT_ID'],df_events['EVENT_CD'],margins=False).head()

EVENT_CD,2,3,4,5,6,8,9,10,11,12,...,14,15,16,17,18,19,20,21,22,23
RESP_BAT_ID,,,,,,,,,,,,,,,,,,,,,
abrej003,311,119,6,2,1,0,7,2,1,1,...,29,6,15,0,6,0,107,43,6,33
adamc001,7,6,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
adaml001,42,37,2,1,0,0,1,0,0,0,...,10,0,1,0,1,1,20,4,1,5
adamm002,162,88,1,1,0,2,1,0,0,1,...,18,5,1,0,0,0,50,22,1,20
adduj002,34,27,1,0,0,0,1,0,0,0,...,10,0,0,0,2,0,11,6,2,1


In [ ]:
df_events[df_events['EVENT_CD'].isin(
    [event_code['SINGLE'],event_code['DOUBLE'],event_code['TRIPLE'],event_code['HR']])]['EVENT_CD'].head()

In [ ]:
df_events[df_events['EVENT_CD'].isin(
    [event_code['SINGLE'],event_code['DOUBLE'],event_code['TRIPLE'],event_code['HR']])]['RESP_BAT_ID'].head()

In [178]:
pd.crosstab(df_events[df_events['EVENT_CD'].isin(
    [event_code['SINGLE'],event_code['DOUBLE'],event_code['TRIPLE'],event_code['HR']])]['RESP_BAT_ID'],
            df_events[df_events['EVENT_CD'].isin(
    [event_code['SINGLE'],event_code['DOUBLE'],event_code['TRIPLE'],event_code['HR']])]['EVENT_CD'],
            margins=False).sort_values(23,ascending=False)

EVENT_CD,20,21,22,23
RESP_BAT_ID,,,,
stanm004,77,32,0,59
judga001,75,24,3,52
martj006,57,26,3,45
davik003,68,28,1,43
gallj002,32,18,3,41
cruzn002,93,28,0,39
bellc002,59,26,4,39
mousm001,89,24,0,38
morrl001,65,22,1,38


In [ ]:
df_events[((flt_home_team['BOS']) | (flt_away_team['BOS'])) & (df_events['EVENT_CD'].isin(
    [event_code['SINGLE'],event_code['DOUBLE'],event_code['TRIPLE'],event_code['HR']]))]['RESP_BAT_ID'].head()

In [ ]:
df_events[((flt_home_team['BOS']) | (flt_away_team['BOS']))].head()

In [ ]:
grp_teams_homers = df_events.groupby([df_events['GAME_ID'].str[0:3]
                                      , df_events['AWAY_TEAM_ID'], df_events['EVENT_CD']==event_code['HR']])

In [ ]:
grp_teams_homers['EVENT_CD'].count()

In [ ]:
grp_away = df_events.groupby(df_events['AWAY_TEAM_ID'])
grp_home = df_events.groupby(df_events['GAME_ID'].str[0:3])

In [ ]:
grp_away['AWAY_TEAM_ID'].count()

In [ ]:
df_events[df_events['EVENT_CD'] == 23]['EVENT_CD']

In [ ]:
df_events[df_events['EVENT_CD'] == 23]['RESP_BAT_ID'].value_counts()

In [ ]:
%matplotlib inline

In [ ]:
grp_home['EVENT_CD'].head()

In [ ]:
grp_hit_type=df_events.groupby('EVENT_CD')

In [ ]:
grp_hit_type.describe()

In [ ]:
df_events[df_events['EVENT_CD' == 23]].groupby('BAT_HOME_ID').min()

In [ ]:
df_events.shape

In [ ]:
%%matplotlib inline